## 📚 שלב 1: ייבוא הספריות

**מה נשתמש:**
- `cv2.QRCodeDetector()` - "מנוע" לזיהוי QR של OpenCV
- `Flask` - לשידור וידאו לטלפון (כמו ב-notebook 2)
- `argparse` - לקבלת פרמטרים מהמשתמש

**מושגים חשובים:**
- **Bounding Box (bbox)** = המלבן שמקיף את ה-QR
- **Aspect Ratio** = יחס רוחב/גובה (ריבוע מושלם = 1.0)
- **Cooldown** = זמן המתנה בין ספירות (למנוע ספירה כפולה)

In [7]:
import os
os.environ["QT_QPA_PLATFORM"] = "xcb"

import cv2
import time
import numpy as np
import threading
from flask import Flask, Response

print("✅ ספריות נטענו בהצלחה!")

✅ ספריות נטענו בהצלחה!


---

## 🎯 שלב 2: זיהוי QR פשוט (ללא פילטרים)

נתחיל עם הזיהוי הכי בסיסי - נראה מה המצלמה מוצאת.

**איך זה עובד:**
1. יוצרים `QRCodeDetector()`
2. קוראים תמונה מהמצלמה
3. `detectAndDecode()` מחפש QR ומנסה לקרוא את התוכן
4. אם מצא - מקבלים: טקסט (תוכן ה-QR) + נקודות (4 פינות של הריבוע)

**נצייר ריבוע סביב ה-QR שנמצא.**

In [8]:
# Best-practice smooth live camera in Jupyter: ipywidgets.Image (no matplotlib redraw)
# Stop the stream by interrupting the cell (Kernel -> Interrupt) or the stop button.

import cv2
import time
import ipywidgets as widgets
from IPython.display import display

# --- camera open (use V4L2 on Raspberry Pi if needed) ---
cap = cv2.VideoCapture(0)  # try: cv2.VideoCapture(0, cv2.CAP_V4L2)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)  # may be ignored, but helps when supported

if not cap.isOpened():
    raise RuntimeError("Cannot open camera")

# warm-up + drop stale frames
for _ in range(20):
    cap.read()

# --- widget output ---
img = widgets.Image(format="jpeg")
display(img)

# --- streaming loop parameters ---
TARGET_FPS = 20                 # widgets can usually handle 15-25 fps smoothly
JPEG_QUALITY = 75               # 60-85 is a good range
DROP_GRABS = 2                  # drop buffered frames to reduce lag
DT = 1.0 / TARGET_FPS

try:
    t_next = time.time()
    while True:
        # drop a couple frames so we always show the newest
        for _ in range(DROP_GRABS):
            cap.grab()

        ret, frame = cap.read()
        if not ret:
            print("failed read")
            time.sleep(0.05)
            continue

        # encode BGR -> JPEG (fast + smooth in notebooks)
        ok, jpg = cv2.imencode(".jpg", frame, [int(cv2.IMWRITE_JPEG_QUALITY), JPEG_QUALITY])
        if ok:
            img.value = jpg.tobytes()

        # pace output (prevents flicker + CPU spikes)
        now = time.time()
        sleep = t_next - now
        if sleep > 0:
            time.sleep(sleep)
        t_next = max(t_next + DT, now)

finally:
    cap.release()


Image(value=b'', format='jpeg')

KeyboardInterrupt: 

---

## 🧪 שלב 3: הוספת פילטרים - בדיקת גודל ומבנה

**הבעיה:** לפעמים המצלמה "רואה" QR שהוא רחוק מאוד או מעוות.  
**הפתרון:** נוסיף תנאים שיבדקו אם ה-QR "טוב" מספיק.

### פילטרים שנשתמש בהם:

1. **גודל מינימלי:**
   - `qr_min_w` = רוחב מינימלי (פיקסלים)
   - `qr_min_h` = גובה מינימלי (פיקסלים)
   - אם ה-QR קטן מדי → דוחים אותו

2. **יחס רוחב-גובה (aspect ratio):**
   - QR אמיתי הוא ריבוע (יחס 1:1)
   - אבל בפרספקטיבה (זווית) זה יכול להיות 0.8 או 1.2
   - נגדיר טווח: `qr_min_aspect` עד `qr_max_aspect`
   - אם היחס מחוץ לטווח → דוחים

**איך נחשב:**
```python
aspect = width / height
if 0.65 <= aspect <= 1.35:  # כמעט ריבוע
    print("✅ זיהוי טוב!")
else:
    print("❌ לא ריבועי מספיק")
```

In [ ]:
# פרמטרים לכיוונון (אפשר לשנות!)
QR_MIN_W = 35        # רוחב מינימלי
QR_MIN_H = 35        # גובה מינימלי
QR_MIN_ASPECT = 0.6  # יחס מינימלי (רוחב/גובה)
QR_MAX_ASPECT = 1.4  # יחס מקסימלי

import cv2
import time
import numpy as np
import ipywidgets as widgets
from IPython.display import display

def passes_filters(points):
    """בודק אם ה-QR עובר את הפילטרים"""
    xs = points[:, 0]
    ys = points[:, 1]
    x1, y1 = int(np.min(xs)), int(np.min(ys))
    x2, y2 = int(np.max(xs)), int(np.max(ys))

    width = x2 - x1
    height = y2 - y1
    aspect = width / max(1, height)  # למנוע חלוקה באפס

    size_ok = (width >= QR_MIN_W) and (height >= QR_MIN_H)
    aspect_ok = (QR_MIN_ASPECT <= aspect <= QR_MAX_ASPECT)

    return size_ok and aspect_ok, width, height, aspect

# Best-practice smooth live camera in Jupyter: ipywidgets.Image (no matplotlib redraw)
# Stop the stream by interrupting the cell (Kernel -> Interrupt) or the stop button.

# --- camera open (use V4L2 on Raspberry Pi if needed) ---
cap = cv2.VideoCapture(0)  # try: cv2.VideoCapture(0, cv2.CAP_V4L2)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)  # may be ignored, but helps when supported

if not cap.isOpened():
    raise RuntimeError("Cannot open camera")

# warm-up + drop stale frames
for _ in range(20):
    cap.read()

detector = cv2.QRCodeDetector()

print("🔬 בודק QR עם פילטרים...")
print(f"פילטר גודל: {QR_MIN_W}x{QR_MIN_H} לפחות")
print(f"פילטר יחס: {QR_MIN_ASPECT:.2f} - {QR_MAX_ASPECT:.2f}")

# --- widget output ---
img = widgets.Image(format="jpeg")
display(img)

# --- streaming loop parameters ---
TARGET_FPS = 20
JPEG_QUALITY = 75
DROP_GRABS = 2
DT = 1.0 / TARGET_FPS

try:
    t_next = time.time()
    while True:
        # drop buffered frames so we always show the newest
        for _ in range(DROP_GRABS):
            cap.grab()

        ret, frame = cap.read()
        if not ret:
            print("failed read")
            time.sleep(0.05)
            continue

        text, points, _ = detector.detectAndDecode(frame)

        if points is not None:
            pts = points[0].astype(int)
            okf, w, h, aspect = passes_filters(points[0])

            color = (0, 255, 0) if okf else (0, 0, 255)
            cv2.polylines(frame, [pts], True, color, 2)

            status = "✅ GOOD" if okf else "❌ BAD"
            info = f"{status} | {w}x{h} | aspect={aspect:.2f}"
            cv2.putText(frame, info, (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

            # optionally show decoded text when filter passes
            if okf and text:
                cv2.putText(frame, text, (10, 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        # encode BGR -> JPEG for notebook display
        ok, jpg = cv2.imencode(".jpg", frame, [int(cv2.IMWRITE_JPEG_QUALITY), JPEG_QUALITY])
        if ok:
            img.value = jpg.tobytes()

        # pace output
        now = time.time()
        sleep = t_next - now
        if sleep > 0:
            time.sleep(sleep)
        t_next = max(t_next + DT, now)

finally:
    cap.release()


---

## 🧮 שלב 4: ספירת זיהויים (עם Cooldown)

**הבעיה:** אם המצלמה רואה אותו QR 30 פעם בשנייה, נספור אותו 30 פעם?  
**הפתרון:** **Cooldown** = זמן המתנה בין ספירות.

### איך זה עובד:
1. כשמוצאים QR טוב → בודקים מתי ספרנו בפעם האחרונה
2. אם עברו יותר מ-`COOLDOWN_MS` מילישניות → סופרים!
3. אחרת → מתעלמים (זה אותו QR)

**דוגמה:**
- `COOLDOWN_MS = 600` → סופרים פעם ב-0.6 שניות לכל היותר
- `COOLDOWN_MS = 2000` → סופרים פעם ב-2 שניות

In [8]:
# Show live screen in Jupyter with QR box + cooldown counter (ipywidgets.Image)
# Stop: interrupt the cell (Kernel -> Interrupt)

import cv2, time, numpy as np
import ipywidgets as widgets
from IPython.display import display

# פרמטרים
QR_MIN_W = 35
QR_MIN_H = 35
QR_MIN_ASPECT = 0.6
QR_MAX_ASPECT = 1.4
COOLDOWN_MS = 600  # 0.6 שניות

def passes_filters(points):
    xs = points[:, 0]
    ys = points[:, 1]
    x1, y1 = int(np.min(xs)), int(np.min(ys))
    x2, y2 = int(np.max(xs)), int(np.max(ys))
    width = x2 - x1
    height = y2 - y1
    aspect = width / max(1, height)
    size_ok = (width >= QR_MIN_W) and (height >= QR_MIN_H)
    aspect_ok = (QR_MIN_ASPECT <= aspect <= QR_MAX_ASPECT)
    return size_ok and aspect_ok, width, height, aspect

accepted_count = 0
last_count_time = 0.0
cooldown_sec = COOLDOWN_MS / 1000.0

# --- camera open (use V4L2 on Raspberry Pi if needed) ---
cap = cv2.VideoCapture(0)  # try: cv2.VideoCapture(0, cv2.CAP_V4L2)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

if not cap.isOpened():
    raise RuntimeError("Cannot open camera")

# warm-up
for _ in range(20):
    cap.read()

detector = cv2.QRCodeDetector()

print(f"📊 מונה QR עם cooldown של {COOLDOWN_MS}ms")

# --- widget output ---
img = widgets.Image(format="jpeg")
display(img)

# streaming params
TARGET_FPS = 20
JPEG_QUALITY = 75
DROP_GRABS = 2
DT = 1.0 / TARGET_FPS

try:
    t_next = time.time()
    while True:
        for _ in range(DROP_GRABS):
            cap.grab()

        ret, frame = cap.read()
        if not ret:
            time.sleep(0.05)
            continue

        now = time.time()
        text, points, _ = detector.detectAndDecode(frame)

        if points is not None:
            pts = points[0].astype(int)
            ok, w, h, aspect = passes_filters(points[0])

            color = (0, 255, 0) if ok else (0, 0, 255)
            cv2.polylines(frame, [pts], True, color, 2)

            # Count only if passes filters + cooldown
            if ok and (now - last_count_time) >= cooldown_sec:
                accepted_count += 1
                last_count_time = now
                print(f"🎯 ספירה #{accepted_count} | text={text!r}")

            # Optional: show decoded text on screen
            if text:
                cv2.putText(frame, text, (10, 80),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        # Counter overlay
        cv2.putText(frame, f"Accepted: {accepted_count}", (10, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 255), 3)

        # Send to notebook
        ok_enc, jpg = cv2.imencode(".jpg", frame, [int(cv2.IMWRITE_JPEG_QUALITY), JPEG_QUALITY])
        if ok_enc:
            img.value = jpg.tobytes()

        # pace
        now2 = time.time()
        sleep = t_next - now2
        if sleep > 0:
            time.sleep(sleep)
        t_next = max(t_next + DT, now2)

finally:
    cap.release()


📊 מונה QR עם cooldown של 600ms


Image(value=b'', format='jpeg')

🎯 ספירה #1 | text=''
🎯 ספירה #2 | text=''
🎯 ספירה #3 | text=''
🎯 ספירה #4 | text=''
🎯 ספירה #5 | text='QR_CAM_01'
🎯 ספירה #6 | text=''
🎯 ספירה #7 | text=''
🎯 ספירה #8 | text='QR_CAM_01'
🎯 ספירה #9 | text=''
🎯 ספירה #10 | text=''
🎯 ספירה #11 | text='QR_CAM_01'
🎯 ספירה #12 | text='QR_CAM_01'
🎯 ספירה #13 | text=''
🎯 ספירה #14 | text='QR_CAM_01'
🎯 ספירה #15 | text='QR_CAM_01'
🎯 ספירה #16 | text='QR_CAM_01'
🎯 ספירה #17 | text=''
🎯 ספירה #18 | text=''
🎯 ספירה #19 | text='QR_CAM_01'
🎯 ספירה #20 | text='QR_CAM_01'
🎯 ספירה #21 | text=''
🎯 ספירה #22 | text=''
🎯 ספירה #23 | text=''
🎯 ספירה #24 | text=''
🎯 ספירה #25 | text=''
🎯 ספירה #26 | text=''
🎯 ספירה #27 | text=''
🎯 ספירה #28 | text='QR_CAM_01'
🎯 ספירה #29 | text='QR_CAM_01'
🎯 ספירה #30 | text=''
🎯 ספירה #31 | text='QR_CAM_01'


KeyboardInterrupt: 

---

## 🎯 ניסוי כיוונון: מצא את הפרמטרים הטובים ביותר

**המשימה:**  
מצאו את הפרמטרים שיאפשרו לרובוט לזהות QR מרחוק, אבל לא "להשתגע" מזיהויים שגויים.

### כיצד לכוונן:

1. **הדפיסו QR על דף A4** (אפשר ליצור כאן: https://www.qr-code-generator.com/)

2. **התחילו קרוב למצלמה:**
   - ה-QR צריך להיות ירוק (מזוהה טוב)
   - רשמו את הגודל: כמה פיקסלים רוחב/גובה?

3. **התרחקו לאט:**
   - באיזה מרחק ה-QR הופך לאדום (נדחה)?
   - באיזה מרחק הזיהוי מפסיק לגמרי?

4. **התאימו את הפרמטרים:**
   - אם ה-QR נדחה מוקדם מדי → הקטינו `QR_MIN_W` ו-`QR_MIN_H`
   - אם זיהויים שגויים עוברים → הגדילו את הפרמטרים
   - אם ה-QR מוטה (לא ריבועי) → הרחיבו את `QR_MIN_ASPECT/MAX_ASPECT`

### טבלת ערכים מומלצים:

| סוג שימוש | MIN_W | MIN_H | MIN_ASPECT | MAX_ASPECT | הסבר |
|-----------|-------|-------|------------|------------|------|
| **קפדני** | 80 | 80 | 0.90 | 1.10 | רק QR גדולים וישרים |
| **סטנדרטי** | 50 | 50 | 0.75 | 1.30 | איזון טוב |
| **מקל** | 30 | 30 | 0.60 | 1.50 | מקבל גם QR קטנים ומעוותים |
| **רחוק** | 20 | 20 | 0.50 | 2.00 | לזיהוי ממרחק (יותר false positives) |

In [ ]:
# 🧪 ניסוי: שנו את הערכים ונסו!

# נסו תחילה את "סטנדרטי"
QR_MIN_W = 50
QR_MIN_H = 50
QR_MIN_ASPECT = 0.75
QR_MAX_ASPECT = 1.30
COOLDOWN_MS = 600

print("🔧 פרמטרים נוכחיים:")
print(f"   גודל מינימלי: {QR_MIN_W}x{QR_MIN_H}")
print(f"   יחס: {QR_MIN_ASPECT:.2f} - {QR_MAX_ASPECT:.2f}")
print(f"   Cooldown: {COOLDOWN_MS}ms")
print()
print("💡 טיפ: שנו את הערכים בתא הזה והריצו מחדש!")

# [הקוד זהה לתא הקודם - העתיקו אותו או קראו לפונקציה]

---

## 📊 סיכום - פקודות ופרמטרים

### פקודות זיהוי QR:

| פקודה | תיאור | פלט |
|-------|--------|-----|
| `detector = cv2.QRCodeDetector()` | יצירת זיהוי QR | אובייקט detector |
| `text, points, _ = detector.detectAndDecode(frame)` | מחפש QR בתמונה | (טקסט, נקודות, ...) |
| `cv2.polylines(frame, [points], True, color, 2)` | מצייר קווים | - |

### פרמטרי כיוונון:

| פרמטר | טווח | ברירת מחדל | השפעה |
|--------|------|------------|--------|
| `QR_MIN_W` | 20-100 | 35 | רוחב מינימלי (פיקסלים) - נמוך = מזהה מרחוק |
| `QR_MIN_H` | 20-100 | 35 | גובה מינימלי (פיקסלים) |
| `QR_MIN_ASPECT` | 0.5-0.95 | 0.6 | יחס מינימלי - נמוך = מקבל יותר עיוותים |
| `QR_MAX_ASPECT` | 1.05-2.0 | 1.4 | יחס מקסימלי |
| `COOLDOWN_MS` | 100-3000 | 600 | זמן בין ספירות (מילישניות) |

### חישובים:

```python
# מציאת bbox מנקודות
x1 = min(points[:, 0])
x2 = max(points[:, 0])
y1 = min(points[:, 1])
y2 = max(points[:, 1])

# חישוב גודל
width = x2 - x1
height = y2 - y1

# חישוב aspect ratio
aspect = width / height
```

---

## 💡 רעיונות למשימות נוספות

1. **מונה QR שונים:**
   - שמרו את הטקסט של כל QR
   - ספרו כמה QR שונים סרקתם

2. **היסטוגרמת מרחקים:**
   - מדדו את גודל ה-QR בכל פעם
   - צרו גרף: "באיזה מרחק זיהינו הכי הרבה?"

3. **משחק צייד QR:**
   - הדביקו 5 QR בחדר
   - כל QR עם טקסט שונה
   - המשימה: למצוא את כולם!

4. **QR מסתובב:**
   - סובבו את ה-QR (0°, 45°, 90°)
   - בדקו אם הזיהוי עדיין עובד
   - מה קורה ל-aspect ratio?

---

## ⚠️ טיפים לפתרון בעיות

| בעיה | פתרון |
|------|-------|
| "לא מזהה בכלל" | בדקו שה-QR חד (לא מטושטש). נסו להגדיל את ה-QR על הנייר |
| "מזהה אדום כל הזמן" | הקטינו את `MIN_W/MIN_H` או הרחיבו את `MIN/MAX_ASPECT` |
| "סופר את אותו QR פעמיים" | הגדילו את `COOLDOWN_MS` |
| "לא סופר בכלל" | בדקו שהזיהוי ירוק. אם כן - הקטינו את `COOLDOWN_MS` |
| "מזהה דברים שהם לא QR" | הגדילו את `MIN_W/MIN_H` והקטינו את טווח ה-aspect |

**בהצלחה! 🎯**